# 爬虫代码实现笔记（三）：数据整理并保存到数据库

为了能够方便地调用笔记二中定义的那些网络爬虫函数，我已将它们整理为一个python函数库：MyReptileTool.py;我们一旦需要用到这些函数，直接导入这库就行了：

In [1]:
import MyReptileTool as mr

用上次笔记中循环迭代的方法，爬取新浪国内新闻版块的新闻：

In [2]:
#将下行url字符中的page参数值设为{},使之成为一个格式字符串
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1531630214332'
news_total=[] #用于保存所有列表页上抓取的新闻正文信息
for i in range(1,3): #抓取的列表页为2页，可自己更改页数范围
    newsurl=url.format(i) #用i值代入格式字符串中的{}位置
    newsary=mr.parseListLinks(newsurl) #抓取指定列表页上的新闻放入newsary这个列表型变量
    news_total.extend(newsary) #将newsary列表并入news_total列表，这样news_total列表中保存了所有列表页上的新闻正文信息了
print(len(news_total))

44


爬取新闻后，抓取到的新闻信息已经存入到news_total这个列表型变量中了；接下来将news_total用pandas整理出来，并保存到数据库：

## 使用pandas整理数据
<br>pandas可以用二维表格（Data Frame）的形式来保存和处理数据，能极大地方便数据处理；<br>
pandas库如果没有的话，就用pip install pandas安装一下；如果使用的是anaconda的话，就已经内置了：

In [3]:
import pandas

In [4]:
df=pandas.DataFrame(news_total)  #用news_total建立一个Data Frame对象
#print(df)
#df  jupyter notebook对pandas支持进行了优化，可直接以表格方式显式dataframe数据
df.head(6) #head方法可指定显示的行数，这里指定了6行；

,article,comments,dt,editor,newssource,title
0,原标题：制度优势：迎击贸易战的关键底气（望海楼） 10天前，美国挑起了对华贸易战，中国被迫应...,15,2018-07-16 05:41:00,霍宇昂,人民日报海外版,人民日报望海楼:制度优势是迎击贸易战的关键底气
1,原标题：中方被迫采取反制行动完全正当合理合法 商务部在声明中说，中方坚持不打第一枪，但在美方...,0,2018-07-16 04:18:00,霍宇昂,人民日报,人民日报:中方被迫采取反制行动完全正当合理合法
2,原标题：泥石流致川藏公路318国道西藏仁布段交通中断 中新网拉萨7月16日电 （何蓬磊 赵延...,0,2018-07-16 01:16:00,霍宇昂,中国新闻网,泥石流致川藏公路318国道西藏仁布段交通中断(图)
3,据泰国《世界日报》报道，泰南酒店业协会会长功萨14日透露，受到船难事件影响，泰南酒店业协会1...,1135,2018-07-16 01:11:00,霍宇昂,环球时报,中国游客取消7300间在7月8月普吉酒店预订房间
4,7月16日凌晨，法国队成功夺得2018年俄罗斯世界杯冠军！正如华帝之前承诺：法国队夺冠，华帝...,174,2018-07-16 01:13:00,霍宇昂,第一财经,法国队夺冠 华帝 “退全款” 正式启动
5,原标题：台军政战宣传部门发文：要认清“共军”还是敌人 近日，有台湾深绿媒体报道，台“国防部政...,76,2018-07-15 23:32:00,霍宇昂,环球时报,台军政战宣传部门发文：要认清“共军”还是敌人


## 将数据保存至数据库中

### 将数据保存至Excel中：

In [5]:
df.to_excel('news.xlsx') #数据保存至excel

提示：在jupyter中按tab键可以提示，比如这里输出df.to_后，按tab键后就可以看到有哪些输出格式；

在代码所在目录中，打开news.xlsx：<br>
<img src='images/12.jpg' /><br>
说明已成功保存为excel文档。

### 将数据保存在数据库sqlite
<br>sqllite是一款轻量级数据库，无须像mysql,orcale那样需要服务器安装和运行端口,可以轻量化实现sql操作; 

In [6]:
import sqlite3

下面代码是用来保存数据至sqlite数据库，库名为news.sqlite;注意，当news.sqlite已经存在时，connect方法会连接上访库，若不存在，则connect方法会根据库名来新建数据库；

In [7]:
with sqlite3.connect('news.sqlite') as db:
    df.to_sql('news',con=db)  #con参数为连接名

数据保存在sqlite中后，便可以用sql对数据库进行增、删、改、查了，下面演示对数据库进行查询，并将结果置入pandas的DataFrame类型变量中：

In [9]:
with sqlite3.connect('news.sqlite') as db:
    df2=pandas.read_sql_query('select * from news',con=db)  #使用sql语句对news进行查询读取，并将结果放入df2中；

In [11]:
df2.head(6)

,index,article,comments,dt,editor,newssource,title
0,0,原标题：制度优势：迎击贸易战的关键底气（望海楼） 10天前，美国挑起了对华贸易战，中国被迫应...,15,2018-07-16 05:41:00,霍宇昂,人民日报海外版,人民日报望海楼:制度优势是迎击贸易战的关键底气
1,1,原标题：中方被迫采取反制行动完全正当合理合法 商务部在声明中说，中方坚持不打第一枪，但在美方...,0,2018-07-16 04:18:00,霍宇昂,人民日报,人民日报:中方被迫采取反制行动完全正当合理合法
2,2,原标题：泥石流致川藏公路318国道西藏仁布段交通中断 中新网拉萨7月16日电 （何蓬磊 赵延...,0,2018-07-16 01:16:00,霍宇昂,中国新闻网,泥石流致川藏公路318国道西藏仁布段交通中断(图)
3,3,据泰国《世界日报》报道，泰南酒店业协会会长功萨14日透露，受到船难事件影响，泰南酒店业协会1...,1135,2018-07-16 01:11:00,霍宇昂,环球时报,中国游客取消7300间在7月8月普吉酒店预订房间
4,4,7月16日凌晨，法国队成功夺得2018年俄罗斯世界杯冠军！正如华帝之前承诺：法国队夺冠，华帝...,174,2018-07-16 01:13:00,霍宇昂,第一财经,法国队夺冠 华帝 “退全款” 正式启动
5,5,原标题：台军政战宣传部门发文：要认清“共军”还是敌人 近日，有台湾深绿媒体报道，台“国防部政...,76,2018-07-15 23:32:00,霍宇昂,环球时报,台军政战宣传部门发文：要认清“共军”还是敌人
